In [94]:
import pandas as pd
import geopandas as gpd
import xarray as xr
import os
from pyproj import Transformer
from scipy.spatial import cKDTree

ModuleNotFoundError: No module named 'scipy'

In [31]:
print(os.getcwd())

/home/franc/Desktop/ProjIntegrador


In [32]:
apa2017 = pd.read_excel('apa/PM10_2017-01-01_2017-12-31.xlsx', header=0)
apa2018 = pd.read_excel('apa/PM10_2018-01-01_2018-12-31.xlsx', header=0)

In [33]:
print(f"Apa 2017 dimensões: {apa2017.shape}")
print(apa2017.head())

Apa 2017 dimensões: (8760, 61)
        PM10 (µg/m3)\t  Alverca  Anta-Espinho  Arcos  Aveiro  \
0  2017-01-01 00:00:00     45.5           NaN   48.8    86.0   
1  2017-01-01 01:00:00     32.2           NaN   48.0    80.0   
2  2017-01-01 02:00:00     31.4           NaN   46.8    75.0   
3  2017-01-01 03:00:00     26.6           NaN   43.3    73.0   
4  2017-01-01 04:00:00     26.1           NaN   37.3    71.0   

   Avenida da Liberdade  Avintes  Burgães-Santo Tirso  \
0                  69.2      NaN                  NaN   
1                  69.7      NaN                  NaN   
2                  55.4      NaN                  NaN   
3                  53.0      NaN                  NaN   
4                  42.7      NaN                  NaN   

   Cascais - Escola da Cidadela  Cerro  ...  Reboleira  Santa Cruz de Benfica  \
0                          55.0   23.1  ...       75.5                   70.9   
1                          52.2   20.5  ...       66.9                   48.7  

In [34]:
print(f"Apa 2018 dimensões: {apa2018.shape}")
print(apa2018.head())

Apa 2018 dimensões: (8760, 61)
        PM10 (µg/m3)\t  Alverca  Anta-Espinho  Arcos  Aveiro  \
0  2018-01-01 00:00:00     54.4           NaN   55.5    85.0   
1  2018-01-01 01:00:00     41.6           NaN   54.8    96.0   
2  2018-01-01 02:00:00     40.0          35.0   53.6    96.0   
3  2018-01-01 03:00:00     21.9          30.0   48.7    79.0   
4  2018-01-01 04:00:00     20.0          33.0   39.8    79.0   

   Avenida da Liberdade  Avintes  Burgães-Santo Tirso  \
0                  64.9      NaN                 22.0   
1                  68.1      NaN                 43.0   
2                  79.0      NaN                  NaN   
3                  83.4      NaN                  NaN   
4                  79.4     43.0                  NaN   

   Cascais - Escola da Cidadela  Cerro  ...  Reboleira  Santa Cruz de Benfica  \
0                          61.9   23.7  ...       50.0                    NaN   
1                          63.3   27.2  ...       70.2                    NaN  

In [35]:
print(type(apa2017.iloc[0,0]))
apa2017["PM10 (µg/m3)\t"] = pd.to_datetime(apa2017["PM10 (µg/m3)	"], format='%Y-%m-%d %H:%M:%S')
print(type(apa2017.iloc[0,0]))
apa2018["PM10 (µg/m3)\t"] = pd.to_datetime(apa2018["PM10 (µg/m3)	"], format='%Y-%m-%d %H:%M:%S')

<class 'str'>
<class 'pandas._libs.tslibs.timestamps.Timestamp'>


In [36]:
"""IMPORTANTE: EM 2017 A COLUNA DE ERVEDEIRA ESTÁ FORMATADA COMO "object" E NÃO "float64" POR ALGUMA RAZÃO"""

print("2017 numeric columns:", len(apa2017.select_dtypes(include='number').columns.tolist()))
print("2018 numeric columns:", len(apa2018.select_dtypes(include='number').columns.tolist()))
print("2017 localidades:", apa2017.shape[1]-1)
print("2018 localidades:", apa2018.shape[1]-1)
non_numeric_2017 = apa2017.columns.difference(apa2017.select_dtypes(include='number').columns)
print("Non-numeric 2017 columns:", non_numeric_2017)
print(apa2017["Ervedeira"].dtype)

2017 numeric columns: 59
2018 numeric columns: 60
2017 localidades: 60
2018 localidades: 60
Non-numeric 2017 columns: Index(['Ervedeira', 'PM10 (µg/m3)\t'], dtype='object')
object


In [37]:
apa2017["Ervedeira"] = pd.to_numeric(apa2017["Ervedeira"].astype(str).str.strip(), errors='coerce')
print(apa2017["Ervedeira"].dtype)  # Should output: float64


float64


In [38]:
#outubro de 2017 (14-18)
mask = (apa2017["PM10 (µg/m3)\t"] >= pd.to_datetime('2017-10-14')) & (apa2017["PM10 (µg/m3)\t"] < pd.to_datetime('2017-10-19')) #devolve um dicionario de falso ou correto para a condição
Napa2017 = apa2017.loc[mask]
Napa2017 = Napa2017.copy() #só para não haver mutabilidade visto que o dataframe é uma fatia de outro
print(Napa2017)
Napa2017.to_excel("apa/apa2017_14_18outubro.xlsx", index=False)

          PM10 (µg/m3)\t  Alverca  Anta-Espinho  Arcos  Aveiro  \
6864 2017-10-14 00:00:00     47.2           0.0   50.8    34.0   
6865 2017-10-14 01:00:00     42.2           5.0   48.4    37.0   
6866 2017-10-14 02:00:00     43.3          40.0   48.2    33.0   
6867 2017-10-14 03:00:00     45.5          38.0   46.3    29.0   
6868 2017-10-14 04:00:00     46.6          24.0   46.1    28.0   
...                  ...      ...           ...    ...     ...   
6979 2017-10-18 19:00:00      8.9           2.0    8.9    23.0   
6980 2017-10-18 20:00:00      8.3          19.0    8.6    25.0   
6981 2017-10-18 21:00:00      6.7          22.0    5.9    26.0   
6982 2017-10-18 22:00:00      6.9          42.0    5.6    25.0   
6983 2017-10-18 23:00:00      7.8          38.0   12.0    24.0   

      Avenida da Liberdade  Avintes  Burgães-Santo Tirso  \
6864                  82.1     46.0                  1.0   
6865                  69.8     40.0                  0.0   
6866                  68.2 

In [39]:
#junho de 2018 (16-22)
mask = (apa2018["PM10 (µg/m3)\t"] >= pd.to_datetime('2018-06-16')) & (apa2018["PM10 (µg/m3)\t"] < pd.to_datetime('2018-06-23'))
Napa2018 = apa2018.loc[mask]
Napa2018 = Napa2018.copy() #só para não haver mutabilidade visto que o dataframe é uma fatia de outro
print(Napa2018)
Napa2018.to_excel("apa/apa2018_16_23junho.xlsx", index=False)

          PM10 (µg/m3)\t  Alverca  Anta-Espinho  Arcos  Aveiro  \
3984 2018-06-16 00:00:00     18.9          25.0   16.9    20.0   
3985 2018-06-16 01:00:00     17.8           4.0   17.1    19.0   
3986 2018-06-16 02:00:00     10.0           6.0   17.1    19.0   
3987 2018-06-16 03:00:00     10.3          28.0   17.4    17.0   
3988 2018-06-16 04:00:00     12.2          28.0   16.5    18.0   
...                  ...      ...           ...    ...     ...   
4147 2018-06-22 19:00:00      NaN           0.0   24.9    17.0   
4148 2018-06-22 20:00:00      NaN           0.0   25.3    18.0   
4149 2018-06-22 21:00:00      NaN           0.0   31.4    19.0   
4150 2018-06-22 22:00:00      NaN           1.0   32.0    19.0   
4151 2018-06-22 23:00:00      NaN          18.0   34.9    19.0   

      Avenida da Liberdade  Avintes  Burgães-Santo Tirso  \
3984                  29.8     22.0                  0.0   
3985                  26.0     25.0                  0.0   
3986                  25.0 

In [40]:
#a média do dia só é calculada se houver pelo menos 19 valores válidos (≈80%) c.c. = NaN

Napa2017["Data"] = Napa2017["PM10 (µg/m3)\t"].dt.date
Napa2017.to_excel("apa/teste2017.xlsx", index=False)

In [41]:
colunas_numericas = Napa2017.select_dtypes(include='number').columns #se usarmos .mean() no dataframe inteiro dá erro porque a primeira e ultima coluna sao datas e não floats
media2017 = Napa2017.groupby("Data")[colunas_numericas].mean()
print(media2017)

              Alverca  Anta-Espinho      Arcos     Aveiro  \
Data                                                        
2017-10-14  54.300000     42.208333  54.062500  44.791667   
2017-10-15  59.066667     63.333333  49.929167  73.458333   
2017-10-16  64.258333     54.500000  64.600000  93.000000   
2017-10-17  28.808333     17.541667  31.158333  45.000000   
2017-10-18  11.125000      9.458333  10.304167  16.083333   

            Avenida da Liberdade    Avintes  Burgães-Santo Tirso  \
Data                                                               
2017-10-14             61.962500  31.500000             4.222222   
2017-10-15             57.408333  32.000000             1.545455   
2017-10-16             67.820833        NaN            30.600000   
2017-10-17             23.591667  22.571429            23.812500   
2017-10-18             13.220833   9.454545             6.565217   

            Cascais - Escola da Cidadela      Cerro   Chamusca  ...  \
Data                    

In [42]:
obs_diarias = Napa2017.groupby("Data").count()
obs_diarias.to_excel("apa/obs17.xlsx", index=False)
obs_diarias=obs_diarias.iloc[:, 1:]
print(obs_diarias)
mask = obs_diarias >= 19

            Alverca  Anta-Espinho  Arcos  Aveiro  Avenida da Liberdade  \
Data                                                                     
2017-10-14       24            24     24      24                    24   
2017-10-15       24            24     24      24                    24   
2017-10-16       24            24     24      24                    24   
2017-10-17       24            24     24      24                    24   
2017-10-18       24            24     24      24                    24   

            Avintes  Burgães-Santo Tirso  Cascais - Escola da Cidadela  Cerro  \
Data                                                                            
2017-10-14       12                   18                            24     24   
2017-10-15        5                   11                            16     24   
2017-10-16        0                   10                             0     18   
2017-10-17       14                   16                             0     1

In [43]:
print(mask)

            Alverca  Anta-Espinho  Arcos  Aveiro  Avenida da Liberdade  \
Data                                                                     
2017-10-14     True          True   True    True                  True   
2017-10-15     True          True   True    True                  True   
2017-10-16     True          True   True    True                  True   
2017-10-17     True          True   True    True                  True   
2017-10-18     True          True   True    True                  True   

            Avintes  Burgães-Santo Tirso  Cascais - Escola da Cidadela  Cerro  \
Data                                                                            
2017-10-14    False                False                          True   True   
2017-10-15    False                False                         False   True   
2017-10-16    False                False                         False  False   
2017-10-17    False                False                         False   Tru

In [44]:
Fmedia2017 = media2017[mask]
print(Fmedia2017)

              Alverca  Anta-Espinho      Arcos     Aveiro  \
Data                                                        
2017-10-14  54.300000     42.208333  54.062500  44.791667   
2017-10-15  59.066667     63.333333  49.929167  73.458333   
2017-10-16  64.258333     54.500000  64.600000  93.000000   
2017-10-17  28.808333     17.541667  31.158333  45.000000   
2017-10-18  11.125000      9.458333  10.304167  16.083333   

            Avenida da Liberdade   Avintes  Burgães-Santo Tirso  \
Data                                                              
2017-10-14             61.962500       NaN                  NaN   
2017-10-15             57.408333       NaN                  NaN   
2017-10-16             67.820833       NaN                  NaN   
2017-10-17             23.591667       NaN                  NaN   
2017-10-18             13.220833  9.454545             6.565217   

            Cascais - Escola da Cidadela      Cerro   Chamusca  ...  \
Data                           

In [45]:
#Repetir para 2018

Napa2018["Data"] = Napa2018["PM10 (µg/m3)\t"].dt.date
Napa2018.to_excel("apa/teste2018.xlsx", index=False)
colunas_numericas = Napa2018.select_dtypes(include='number').columns #se usarmos .mean() no dataframe inteiro dá erro porque a primeira e ultima coluna sao datas e não floats
media2018 = Napa2018.groupby("Data")[colunas_numericas].mean()
obs_diarias = Napa2018.groupby("Data").count()
obs_diarias.to_excel("apa/obs18.xlsx", index=False)
obs_diarias=obs_diarias.iloc[:, 1:]
mask = obs_diarias >= 19
Fmedia2018 = media2018[mask]

In [46]:
Fmedia2017 = Fmedia2017.replace("", pd.NA)
Fmedia2017["proporção"] = Fmedia2017.notna().sum(axis=1) / (Fmedia2017.shape[1])
print(Fmedia2017)
Fmedia2017.to_excel("apa/Fmedia2017.xlsx")

Fmedia2018 = Fmedia2018.replace("", pd.NA)
Fmedia2018["proporção"] = (Fmedia2018.notna().sum(axis=1) / (Fmedia2018.shape[1]))
print(Fmedia2018)
Fmedia2018.to_excel("apa/Fmedia2018.xlsx")

              Alverca  Anta-Espinho      Arcos     Aveiro  \
Data                                                        
2017-10-14  54.300000     42.208333  54.062500  44.791667   
2017-10-15  59.066667     63.333333  49.929167  73.458333   
2017-10-16  64.258333     54.500000  64.600000  93.000000   
2017-10-17  28.808333     17.541667  31.158333  45.000000   
2017-10-18  11.125000      9.458333  10.304167  16.083333   

            Avenida da Liberdade   Avintes  Burgães-Santo Tirso  \
Data                                                              
2017-10-14             61.962500       NaN                  NaN   
2017-10-15             57.408333       NaN                  NaN   
2017-10-16             67.820833       NaN                  NaN   
2017-10-17             23.591667       NaN                  NaN   
2017-10-18             13.220833  9.454545             6.565217   

            Cascais - Escola da Cidadela      Cerro   Chamusca  ...  \
Data                           

In [47]:
gdf = gpd.read_file("estacoes/qualar_estacao_inspire_vw.shp")
print(gdf.columns.tolist())
print(f"Dimensão da info geofŕafica das estações: {gdf.shape}\n")
print(gdf.head())

['codigo_est', 'nome_estac', 'data_inici', 'data_fim', 'latitude', 'longitude', 'rede_codig', 'rede_nome', 'rede_aqd_o', 'geometry']
Dimensão da info geofŕafica das estações: (65, 10)

    codigo_est                      nome_estac  data_inici    data_fim  \
0  STA-PT01053                         Avintes  12/07/2010  0000/00/00   
1  STA-PT01021             Custóias-Matosinhos  01/09/1998  0000/00/00   
2  STA-PT01023               Ermesinde-Valongo  01/10/1998  0000/00/00   
3  STA-PT01024             D.Manuel II-Vermoim  01/11/1999  0000/00/00   
4  STA-PT01028  Francisco Sá Carneiro-Campanha  19/09/2000  0000/00/00   

    latitude  longitude   rede_codig                         rede_nome  \
0  41.098187  -8.554351  NET-PT00015  Rede de Qualidade do Ar do Norte   
1  41.199805  -8.644844  NET-PT00015  Rede de Qualidade do Ar do Norte   
2  41.206605  -8.552600  NET-PT00015  Rede de Qualidade do Ar do Norte   
3  41.235563  -8.618715  NET-PT00015  Rede de Qualidade do Ar do Norte   


In [48]:
cams2017 = xr.open_dataset("cams/PM10_daymean_2017.nc")
print(cams2017)

<xarray.Dataset> Size: 4MB
Dimensions:    (lon: 40, lat: 66, time: 365, bnds: 2)
Coordinates:
  * lon        (lon) float64 320B -10.0 -9.9 -9.8 -9.7 ... -6.4 -6.3 -6.2 -6.1
  * lat        (lat) float64 528B 36.5 36.6 36.7 36.8 ... 42.7 42.8 42.9 43.0
  * time       (time) datetime64[ns] 3kB 2017-01-01T11:30:00 ... 2017-12-31T1...
Dimensions without coordinates: bnds
Data variables:
    time_bnds  (time, bnds) datetime64[ns] 6kB ...
    pm10       (time, lat, lon) float32 4MB ...
Attributes:
    CDI:          Climate Data Interface version ?? (http://mpimet.mpg.de/cdi)
    Conventions:  CF-1.7
    history:      Fri Apr 11 12:39:22 2025: cdo daymean PM10_2017.nc PM10_day...
    Title:        CAMS European air quality interim reanalysis
    Provider:     COPERNICUS European air quality service
    Production:   COPERNICUS Atmosphere Monitoring Service
    CDO:          Climate Data Operators version 1.7.0 (http://mpimet.mpg.de/...


In [49]:
print(cams2017.data_vars)
print("---------------------------------------\n")
print(cams2017.coords)
print("---------------------------------------\n")
print(cams2017["pm10"])
print("---------------------------------------")

Data variables:
    time_bnds  (time, bnds) datetime64[ns] 6kB ...
    pm10       (time, lat, lon) float32 4MB ...
---------------------------------------

Coordinates:
  * lon      (lon) float64 320B -10.0 -9.9 -9.8 -9.7 ... -6.4 -6.3 -6.2 -6.1
  * lat      (lat) float64 528B 36.5 36.6 36.7 36.8 36.9 ... 42.7 42.8 42.9 43.0
  * time     (time) datetime64[ns] 3kB 2017-01-01T11:30:00 ... 2017-12-31T11:...
---------------------------------------

<xarray.DataArray 'pm10' (time: 365, lat: 66, lon: 40)> Size: 4MB
[963600 values with dtype=float32]
Coordinates:
  * lon      (lon) float64 320B -10.0 -9.9 -9.8 -9.7 ... -6.4 -6.3 -6.2 -6.1
  * lat      (lat) float64 528B 36.5 36.6 36.7 36.8 36.9 ... 42.7 42.8 42.9 43.0
  * time     (time) datetime64[ns] 3kB 2017-01-01T11:30:00 ... 2017-12-31T11:...
Attributes:
    standard_name:  mass_concentration_of_pm10_ambient_aerosol_in_air
    long_name:      mass concentration of particulate matter with d < 10 µm
    units:          µg/m3
--------------

![1 de janeiro de 2017](imagens/1_janeiro_2017.png)
![15 de outubro de 2017](imagens/15_outubro_2017.png)
![19 de junho de 2018](imagens/19_junho_2018.png)

In [50]:
pm10 = cams2017["pm10"]
cams2017df = pm10.to_dataframe().reset_index()
print(cams2017df.columns.tolist())
print(f"Dimensão dos dados das camaras: {cams2017df.shape}\n")
print(cams2017df.head())
excerto2017 = cams2017df.iloc[:len(cams2017df)//10] #para ver o que se passava é mais fácil ver somente um décimo do dataset visto que é enorme
excerto2017.to_excel("cams/pm10cams2017.xlsx", index=False)


['time', 'lat', 'lon', 'pm10']
Dimensão dos dados das camaras: (963600, 4)

                 time   lat   lon       pm10
0 2017-01-01 11:30:00  36.5 -10.0  18.154917
1 2017-01-01 11:30:00  36.5  -9.9  17.517908
2 2017-01-01 11:30:00  36.5  -9.8  16.446743
3 2017-01-01 11:30:00  36.5  -9.7  16.456955
4 2017-01-01 11:30:00  36.5  -9.6  16.954844


1) Atribuir coordenadas às localidades no dataframe com os valores médios limpos
2) É preciso projeção de coordenadas ou os dados das cams e apa estão ampas no mesmo tipo???? 
    caso não: arredondar visto que nas netCDFs a latitude e longitude estão arredondadas ---> pode levar a problemas depois na comparação
3) Juntar dados num unico dataset (2 no nosso caso, 1 para cada): 
    ordem cronologica
    pm10 apa e pm10 cams
    localidade com as respetivas coordenadas

In [ ]:
transformer = Transformer.from_crs("EPSG:4326", "EPSG:3763", always_xy=True)
gdf.to_excel("apa/gdf1.xlsx", index=False)
x, y = transformer.transform(gdf['longitude'].values, gdf['latitude'].values)

gdf["3763_long"] = x
gdf["3763_lat"] = y
print(gdf.head())
gdf.to_excel("apa/gdf2.xlsx", index=False)

    codigo_est                      nome_estac  data_inici    data_fim  \
0  STA-PT01053                         Avintes  12/07/2010  0000/00/00   
1  STA-PT01021             Custóias-Matosinhos  01/09/1998  0000/00/00   
2  STA-PT01023               Ermesinde-Valongo  01/10/1998  0000/00/00   
3  STA-PT01024             D.Manuel II-Vermoim  01/11/1999  0000/00/00   
4  STA-PT01028  Francisco Sá Carneiro-Campanha  19/09/2000  0000/00/00   

    latitude  longitude   rede_codig                         rede_nome  \
0  41.098187  -8.554351  NET-PT00015  Rede de Qualidade do Ar do Norte   
1  41.199805  -8.644844  NET-PT00015  Rede de Qualidade do Ar do Norte   
2  41.206605  -8.552600  NET-PT00015  Rede de Qualidade do Ar do Norte   
3  41.235563  -8.618715  NET-PT00015  Rede de Qualidade do Ar do Norte   
4  41.162791  -8.590847  NET-PT00015  Rede de Qualidade do Ar do Norte   

  rede_aqd_o                   geometry     3763_long       3763_lat  
0   regional  POINT (-8.55435 41.09819)

In [52]:
x1, y1 = transformer.transform(cams2017df['lon'].values, cams2017df['lat'].values)

cams2017df.loc[:, "3763_long"] = x1
cams2017df.loc[:, "3763_lat"] = y1

print(cams2017df.head())


                 time   lat   lon       pm10      3763_long       3763_lat
0 2017-01-01 11:30:00  36.5 -10.0  18.154917 -167265.612117 -350049.532044
1 2017-01-01 11:30:00  36.5  -9.9  17.517908 -158305.172793 -350218.603496
2 2017-01-01 11:30:00  36.5  -9.8  16.446743 -149344.875659 -350378.361097
3 2017-01-01 11:30:00  36.5  -9.7  16.456955 -140384.712685 -350528.806258
4 2017-01-01 11:30:00  36.5  -9.6  16.954844 -131424.675835 -350669.940311


In [53]:
cams2018 = xr.open_dataset("cams/PM10_daymean_2018.nc")
pm10 = cams2018["pm10"]
cams2018df = pm10.to_dataframe().reset_index()
print(cams2018df.columns.tolist())
print(f"Dimensão dos dados das camaras: {cams2018df.shape}\n")
print(cams2018df.head())

['time', 'lat', 'lon', 'pm10']
Dimensão dos dados das camaras: (963600, 4)

                 time   lat   lon       pm10
0 2018-01-01 11:30:00  36.5 -10.0  22.185546
1 2018-01-01 11:30:00  36.5  -9.9  22.135416
2 2018-01-01 11:30:00  36.5  -9.8  22.132486
3 2018-01-01 11:30:00  36.5  -9.7  22.175129
4 2018-01-01 11:30:00  36.5  -9.6  22.148437


In [54]:
x1, y1 = transformer.transform(cams2018df['lon'].values, cams2018df['lat'].values)

cams2018df.loc[:, "3763_long"] = x1
cams2018df.loc[:, "3763_lat"] = y1

print(cams2018df.head())

                 time   lat   lon       pm10      3763_long       3763_lat
0 2018-01-01 11:30:00  36.5 -10.0  22.185546 -167265.612117 -350049.532044
1 2018-01-01 11:30:00  36.5  -9.9  22.135416 -158305.172793 -350218.603496
2 2018-01-01 11:30:00  36.5  -9.8  22.132486 -149344.875659 -350378.361097
3 2018-01-01 11:30:00  36.5  -9.7  22.175129 -140384.712685 -350528.806258
4 2018-01-01 11:30:00  36.5  -9.6  22.148437 -131424.675835 -350669.940311


In [55]:
print(type(cams2017df.iloc[0,0]))
cams2017df["time"] = pd.to_datetime(cams2017df["time"], format='%Y-%m-%d %H:%M:%S')
print(type(cams2017df.iloc[0,0]))
cams2018df["time"] = pd.to_datetime(cams2018df["time"], format='%Y-%m-%d %H:%M:%S')

<class 'pandas._libs.tslibs.timestamps.Timestamp'>
<class 'pandas._libs.tslibs.timestamps.Timestamp'>


In [56]:
#outubro de 2017 (14-18)
mask = (cams2017df["time"] >= pd.to_datetime('2017-10-14')) & (cams2017df["time"] < pd.to_datetime('2017-10-19')) #devolve um dicionario de falso ou correto para a condição
camaras2017 = cams2017df.loc[mask]
camaras2017 = camaras2017.copy() #só para não haver mutabilidade visto que o dataframe é uma fatia de outro
print(camaras2017)
camaras2017.to_excel("cams/cams2017_14_18outubro.xlsx", index=False)

                      time   lat   lon       pm10      3763_long  \
755040 2017-10-14 11:30:00  36.5 -10.0  42.453941 -167265.612117   
755041 2017-10-14 11:30:00  36.5  -9.9  42.392410 -158305.172793   
755042 2017-10-14 11:30:00  36.5  -9.8  42.162560 -149344.875659   
755043 2017-10-14 11:30:00  36.5  -9.7  41.917164 -140384.712685   
755044 2017-10-14 11:30:00  36.5  -9.6  42.237843 -131424.675835   
...                    ...   ...   ...        ...            ...   
768235 2017-10-18 11:30:00  43.0  -6.5  16.272829  133166.524410   
768236 2017-10-18 11:30:00  43.0  -6.4  13.677632  141320.872736   
768237 2017-10-18 11:30:00  43.0  -6.3  12.745512  149475.251732   
768238 2017-10-18 11:30:00  43.0  -6.2  12.116454  157629.663144   
768239 2017-10-18 11:30:00  43.0  -6.1  12.044427  165784.108715   

             3763_lat  
755040 -350049.532044  
755041 -350218.603496  
755042 -350378.361097  
755043 -350528.806258  
755044 -350669.940311  
...               ...  
768235  371319.

In [57]:
#junho de 2018 (16-22)
mask = (cams2018df["time"] >= pd.to_datetime('2018-06-16')) & (cams2018df["time"] < pd.to_datetime('2018-06-23')) #devolve um dicionario de falso ou correto para a condição
camaras2018 = cams2018df.loc[mask]
camaras2018 = camaras2018.copy() #só para não haver mutabilidade visto que o dataframe é uma fatia de outro
print(camaras2018)
camaras2018.to_excel("cams/cams2018_16_22junho.xlsx", index=False)

                      time   lat   lon       pm10      3763_long  \
438240 2018-06-16 11:30:00  36.5 -10.0  27.978189 -167265.612117   
438241 2018-06-16 11:30:00  36.5  -9.9  27.848306 -158305.172793   
438242 2018-06-16 11:30:00  36.5  -9.8  27.857421 -149344.875659   
438243 2018-06-16 11:30:00  36.5  -9.7  27.400064 -140384.712685   
438244 2018-06-16 11:30:00  36.5  -9.6  27.009765 -131424.675835   
...                    ...   ...   ...        ...            ...   
456715 2018-06-22 11:30:00  43.0  -6.5  11.020507  133166.524410   
456716 2018-06-22 11:30:00  43.0  -6.4  10.670572  141320.872736   
456717 2018-06-22 11:30:00  43.0  -6.3  10.260986  149475.251732   
456718 2018-06-22 11:30:00  43.0  -6.2  10.341471  157629.663144   
456719 2018-06-22 11:30:00  43.0  -6.1  10.629801  165784.108715   

             3763_lat  
438240 -350049.532044  
438241 -350218.603496  
438242 -350378.361097  
438243 -350528.806258  
438244 -350669.940311  
...               ...  
456715  371319.

In [ ]:
gdf_min = gdf[['nome_estac', '3763_long', '3763_lat']]
print(gdf_min.head())

                       nome_estac     3763_long       3763_lat
0                         Avintes -35388.726933  158867.677319
1             Custóias-Matosinhos -42924.770151  170193.796239
2               Ermesinde-Valongo -35183.611562  170907.573844
3             D.Manuel II-Vermoim -40710.864377  174152.466336
4  Francisco Sá Carneiro-Campanha -38417.071842  166057.858413


In [62]:
print(camaras2017.head())
cam2017_estac =  camaras2017.merge(gdf_min, on=['3763_lat', '3763_long'], how='left')
print(cam2017_estac.head())

                      time   lat   lon       pm10      3763_long  \
755040 2017-10-14 11:30:00  36.5 -10.0  42.453941 -167265.612117   
755041 2017-10-14 11:30:00  36.5  -9.9  42.392410 -158305.172793   
755042 2017-10-14 11:30:00  36.5  -9.8  42.162560 -149344.875659   
755043 2017-10-14 11:30:00  36.5  -9.7  41.917164 -140384.712685   
755044 2017-10-14 11:30:00  36.5  -9.6  42.237843 -131424.675835   

             3763_lat  
755040 -350049.532044  
755041 -350218.603496  
755042 -350378.361097  
755043 -350528.806258  
755044 -350669.940311  
                 time   lat   lon       pm10      3763_long       3763_lat  \
0 2017-10-14 11:30:00  36.5 -10.0  42.453941 -167265.612117 -350049.532044   
1 2017-10-14 11:30:00  36.5  -9.9  42.392410 -158305.172793 -350218.603496   
2 2017-10-14 11:30:00  36.5  -9.8  42.162560 -149344.875659 -350378.361097   
3 2017-10-14 11:30:00  36.5  -9.7  41.917164 -140384.712685 -350528.806258   
4 2017-10-14 11:30:00  36.5  -9.6  42.237843 -131424.6758

In [66]:
print(cam2017_estac["nome_estac"].unique())

"""NÃO DEU"""

[nan]


'NÃO DEU'

In [92]:
gdf_min = gdf[['nome_estac', 'longitude', 'latitude']]
gdf_min = gdf_min.rename(columns={
    'latitude': 'lat',
    'longitude': 'lon'
})
cam2017_estac =  camaras2017.merge(gdf_min, on=['lat', 'lon'], how='left')
print(cam2017_estac.head())
print(cam2017_estac["nome_estac"].unique())

"""NÃO DEU"""


                 time   lat   lon       pm10      3763_long       3763_lat  \
0 2017-10-14 11:30:00  36.5 -10.0  42.453941 -167265.612117 -350049.532044   
1 2017-10-14 11:30:00  36.5  -9.9  42.392410 -158305.172793 -350218.603496   
2 2017-10-14 11:30:00  36.5  -9.8  42.162560 -149344.875659 -350378.361097   
3 2017-10-14 11:30:00  36.5  -9.7  41.917164 -140384.712685 -350528.806258   
4 2017-10-14 11:30:00  36.5  -9.6  42.237843 -131424.675835 -350669.940311   

  nome_estac  
0        NaN  
1        NaN  
2        NaN  
3        NaN  
4        NaN  
[nan]


'NÃO DEU'

In [77]:
print(camaras2017[['3763_lat', '3763_long']].drop_duplicates())
print("nº de coordenadas unicas:", len(camaras2017[['3763_lat', '3763_long']].drop_duplicates()))
print("entries no df inteiro:", camaras2017.shape[0])
print(5 * 2640)

             3763_lat      3763_long
755040 -350049.532044 -167265.612117
755041 -350218.603496 -158305.172793
755042 -350378.361097 -149344.875659
755043 -350528.806258 -140384.712685
755044 -350669.940311 -131424.675835
...               ...            ...
757675  371319.110964  133166.524410
757676  371482.524421  141320.872736
757677  371655.653689  149475.251732
757678  371838.499941  157629.663144
757679  372031.064418  165784.108715

[2640 rows x 2 columns]
nº de coordenadas unicas: 2640
entries no df inteiro: 13200
13200


In [76]:
print(camaras2018[['3763_lat', '3763_long']].drop_duplicates())
print("nº de coordenadas unicas:", len(camaras2018[['3763_lat', '3763_long']].drop_duplicates()))
print("entries no df inteiro:", camaras2018.shape[0])
print(7 * 2640)

             3763_lat      3763_long
438240 -350049.532044 -167265.612117
438241 -350218.603496 -158305.172793
438242 -350378.361097 -149344.875659
438243 -350528.806258 -140384.712685
438244 -350669.940311 -131424.675835
...               ...            ...
440875  371319.110964  133166.524410
440876  371482.524421  141320.872736
440877  371655.653689  149475.251732
440878  371838.499941  157629.663144
440879  372031.064418  165784.108715

[2640 rows x 2 columns]
nº de coordenadas unicas: 2640
entries no df inteiro: 18480
18480


In [83]:
(camaras2017[['3763_lat', '3763_long']].iloc[:-2640].values == camaras2017[['3763_lat', '3763_long']].iloc[2640:].values).all()

np.True_

In [84]:
(camaras2018[['3763_lat', '3763_long']].iloc[:-2640].values == camaras2018[['3763_lat', '3763_long']].iloc[2640:].values).all()

np.True_

In [85]:
"""CONCLUSÃO: coordenadas repetem passado 2640 entradas"""

'CONCLUSÃO: coordenadas repetem passado 2640 entradas'